In [3]:
import pdfplumber
import re
import json
import os
import PyPDF2
import re
import pandas as pd
import sys
import os


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph


In [22]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")


NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)#, enhanced_schema=True)

print(graph.schema)

Node properties:
Tecnica {nome: STRING, Livello: STRING, tipo: STRING, caratteristica: STRING, version: STRING, livello: STRING, uso: STRING, applicazione: STRING, effetto: STRING, cabina: STRING, qualita: STRING, gravita: STRING, descrizione: STRING, Descrizione: STRING, Svantaggi: STRING, Vantaggi: STRING, Funziona: STRING, licenza: STRING, vantaggi: STRING, svantaggi: STRING, funzione: STRING, fondamentali: STRING, funziona: STRING, ingredienti: STRING, moduli_elettro_osmotici: STRING, sequenze_innesto: STRING, distribuzione_idrica: STRING, sfera_onirico_mistica: STRING, matrice_cosmica: STRING, narrazione: STRING, campi_energetici: STRING, calore_residuo: STRING, sincronizzazione_energie: STRING, bioluminescenza: STRING, impulsi_freddi: STRING, condizioni_ambientali: STRING, campo_distorsione_temporale: STRING, campo: STRING, blocco: STRING, ghiaccio: STRING, compartimenti: STRING, flussi: STRING, algoritmi: STRING, stato: STRING, fluttuazioni: STRING, neutralizzazione: STRING, tem

In [14]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)
llm.invoke("Ciao, come posso aiutarti?")

AIMessage(content='Ciao! Grazie per esserti offerto di aiutarmi. Avrei bisogno di assistenza con [specifica richiesta]. Potresti darmi una mano?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1133, 'prompt_tokens': 15, 'total_tokens': 1148, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 1088, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'o1-mini-2024-09-12', 'system_fingerprint': 'fp_f6ff3bb326', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {

In [9]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True
)

In [10]:
chain.invoke({"query": "Who played in Top Gun?"})





> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n) RETURN n LIMIT 0

Full Context:
[]

> Finished chain.


{'query': 'Who played in Top Gun?', 'result': "I don't know the answer."}

# Test 2


In [15]:
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langgraph.graph import StateGraph, END


load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")


NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"


In [5]:
class HybridRetriever:
    def __init__(self, graph, vector_index):
        self.graph = graph  # Neo4jGraph connection
        self.vector = vector_index  # Neo4jVector index
    
    def retrieve(self, question):
        # Vector search first for technique matching
        vector_results = self.vector.similarity_search(
            question, k=3,
            query_type="hybrid",  # From source [3]
            score_threshold=0.25
        )
        
        # Extract technical terms for graph query
        technique_terms = [doc.metadata['name'] for doc in vector_results]
        
        # Graph traversal query
        cypher = f"""
        MATCH (p:Place)-[:USES_TECHNIQUE]->(t:Technique)
        WHERE t.name IN {technique_terms}
        WITH p, COUNT(t) AS technique_match
        MATCH (p)-[:PRODUCES]->(i:Item)
        RETURN p.name, technique_match, COUNT(i) AS total_items
        ORDER BY technique_match DESC
        """
        return self.graph.query(cypher)  # From source [4]

In [6]:
from langgraph.graph import StateGraph, END

class QueryProcessor:
    def __init__(self, llm):
        self.llm = llm
        self.workflow = StateGraph(GraphState)
        
        # Define nodes
        self.workflow.add_node("decompose", self.decompose_query)
        self.workflow.add_node("vector_search", self.run_vector_search)
        self.workflow.add_node("graph_query", self.execute_graph_query)
        
        # Configure edges
        self.workflow.set_entry_point("decompose")
        self.workflow.add_edge("decompose", "vector_search")
        self.workflow.add_conditional_edges(
            "vector_search",
            self.route_based_on_results,
            {"graph": "graph_query", "end": END}
        )
        self.workflow.add_edge("graph_query", END)
    
    def decompose_query(self, state):
        # From source [1]
        decomposition_prompt = ChatPromptTemplate.from_template("""
        Break this into sub-queries: {question}
        Output format: ["subquery1", "subquery2"]
        """)
        chain = decomposition_prompt | self.llm | StrOutputParser()
        return {"subqueries": chain.invoke(state["question"])}
    
    def route_based_on_results(self, state):
        if len(state["vector_results"]) > 0:
            return "graph_query"
        return "end"


In [2]:
from langchain.evaluation import load_evaluator
from langchain.evaluation.agents import TrajectoryEvalChain

# Alternative direct import from source code [4]
from langchain.evaluation.agents.trajectory_eval_chain import (
    TrajectoryEvalChain,
    TrajectoryOutputParser
)

class ValidatedCypherChain:
    def __init__(self, graph, llm):
        self.graph = graph
        self.checker = TrajectoryEvalChain(llm=llm)  # From source [6]
        
    def validate_cypher(self, cypher):
        validation_template = """Check Cypher query for:
        - NULL handling in NOT IN clauses
        - Proper relationship direction
        - Valid property references
        Query: {query}
        """
        return self.checker.invoke(
            validation_template.format(query=cypher)
        )
    
    def safe_query(self, cypher, max_retries=3):
        for _ in range(max_retries):
            validation = self.validate_cypher(cypher)
            if validation["valid"]:
                try:
                    return self.graph.query(cypher)
                except Neo4jError as e:
                    cypher = self._correct_query(cypher, str(e))
        raise ValueError("Max retries exceeded")
    
    def _correct_query(self, original, error):
        correction_prompt = f"""
        Original query: {original}
        Error: {error}
        Provide corrected Cypher:
        """
        return self.llm.invoke(correction_prompt)


In [7]:
from langchain_core.messages import trim_messages

class ContextManager:
    def __init__(self, llm, max_tokens=4000):
        self.llm = llm
        self.max_tokens = max_tokens
        
    def trim_context(self, messages):
        return trim_messages(
            messages,
            token_counter=self.llm.get_message_token_count,
            max_tokens=self.max_tokens,
            strategy="last",
            include_system=True  # From source [5]
        )
    
    def build_prompt(self, question, results):
        context = f"Graph Results: {results}"
        return [
            SystemMessage("You're a technical analyst. Be precise with numbers."),
            HumanMessage(f"Question: {question}\nContext: {context}")
        ]


In [9]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [12]:
evaluator = load_evaluator(
    "trajectory",
    llm=llm,
    agent_tools=[]  # Add your tools here if needed
)

def validate_agent_steps(question, intermediate_steps):
    return evaluator.evaluate(
        inputs={"question": question},
        prediction="",  # Add final agent output
        agent_trajectory=intermediate_steps
    )

In [ ]:
def full_workflow(question):
    # Initialize components
    graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="xxx")
    vector = Neo4jVector.from_existing_graph(...)
    
    # Process
    processor = QueryProcessor(llm)
    context_mgr = ContextManager(llm)
    validator = ValidatedCypherChain(graph, llm)
    
    # Execution
    state = {"question": question}
    state = processor.decompose_query(state)
    state["vector_results"] = HybridRetriever(graph, vector).retrieve(state["subqueries"])
    state["graph_results"] = validator.safe_query(
        construct_cypher(state["subqueries"], state["vector_results"])
    )
    
    # Final answer generation
    trimmed = context_mgr.trim_context(
        context_mgr.build_prompt(question, state["graph_results"])
    )
    return llm.invoke(trimmed)


In [ ]:
class ValidatingChain(GraphCypherQAChain):
    def _call(self, inputs):
        for _ in range(3):  # Max retries
            try:
                cypher = self.generate_cypher(inputs["question"])
                result = self.graph.query(cypher)
                if self.validate_result(result, inputs["question"]):
                    return self.generate_answer(result, inputs["question"])
            except Neo4jError as e:
                cypher = self.refine_cypher(cypher, str(e))


In [ ]:
def answer_with_context(question, graph_results):
    prompt = f"""
    Question: {question}
    Database Results: {graph_results}
    Answer concisely using ONLY the provided data:
    """
    return llm.invoke(prompt)


# TEST 3


In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
import os 
from typing import Any, Dict


load_dotenv()

EMB_OPENAI_API_KEY = os.getenv("EMB_OPENAI_API_KEY")
EMB_OPENAI_ENDPOINT = os.getenv("EMB_OPENAI_ENDPOINT")
EMB_OPENAI_MODEL_NAME = os.getenv("EMB_OPENAI_MODEL_NAME")
EMB_OPENAI_API_VERSION = os.getenv("EMB_OPENAI_API_VERSION")

emb_model = AzureOpenAIEmbeddings(
    model=EMB_OPENAI_MODEL_NAME,
    azure_endpoint=EMB_OPENAI_ENDPOINT,
    openai_api_version=EMB_OPENAI_API_VERSION,
    openai_api_key=EMB_OPENAI_API_KEY
)

# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")


NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)#, enhanced_schema=True)

from langchain_openai import AzureChatOpenAI
import os
from openai import AzureOpenAI

class CustomAzureClient(AzureOpenAI):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def chat(self) -> "CustomChatCompletions":
        return CustomChatCompletions(self._client)
    
"""    def _process_params(self, params: dict) -> dict:
        params.pop('stop', None)
        return super()._process_params(params)
       
    def chat(self, *args, **kwargs):
        # Rimuove il parametro 'stop' a livello di client
        kwargs.pop('stop', None)
        return super().chat(*args, **kwargs)
    
    def completions(self, *args, **kwargs):
        kwargs.pop('stop', None)
        return super().completions(*args, **kwargs)"""

class CustomChatCompletions:
    def __init__(self, client: Any):
        self._client = client
    
    def create(self, **kwargs: Any) -> Any:
        kwargs.pop('stop', None)
        return self._client.chat.completions.create(**kwargs)
    
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    client = CustomAzureClient(
        api_key=api_key,
        api_version="2024-08-01-preview",
        azure_endpoint=api_base
    )

    """return AzureChatOpenAI(
        azure_deployment="gpt-4o",
        #client=client,  # Usa il client personalizzato
        azure_endpoint=f"{api_base}/openai/deployments/gpt-4o/chat/completions",
        temperature=1,
        max_retries=3,
        streaming=False,
        model_kwargs={'stop': None},
        disabled_params={"stop": None},
        http_client=client#CustomAzureClient()
    )"""
    
    return AzureChatOpenAI(
        azure_deployment="gpt-4",#"gpt-4o",
        openai_api_key=api_key,
        azure_endpoint=f"{api_base}/openai/deployments/gpt-4o/chat/completions",
        openai_api_version="2024-08-01-preview",
        temperature=1,
        max_retries=3,
        #http_client=client,
        model_kwargs={'stop': None}
    )
    
llm = crea_llm(api_key, api_base)


c:\Users\smancone\OneDrive - BUSINESS INTEGRATION PARTNERS SPA\Desktop\progetti\2402_PRYSMIAN\.conda\Lib\site-packages\IPython\core\interactiveshell.py:3577: UserWarning: Parameters {'stop'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
test_client = CustomAzureClient(
    api_key=api_key,
    api_version="2024-08-01-preview",
    azure_endpoint=api_base
)

# Test diretto senza LangChain
response = test_client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "Hello"}]
    )

In [21]:
llm.invoke("ciao")

AIMessage(content='Ciao! Come posso aiutarti oggi?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 10, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 64, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'o1-mini-2024-09-12', 'system_fingerprint': 'fp_f6ff3bb326', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': F

In [ ]:
from langchain_openai import AzureChatOpenAI
from openai import AzureOpenAI
from typing import Optional
from langchain_core.language_models.llms import BaseLLM


class CustomAzureClient(AzureOpenAI):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def _process_params(self, params: dict) -> dict:
        """Rimuove definitivamente il parametro stop da tutte le chiamate"""
        params.pop('stop', None)
        params.pop('stop_sequences', None)
        return super()._process_params(params)

class FixedAzureChatOpenAI(AzureChatOpenAI):
    def bind(self, **kwargs) -> BaseLLM:
        """Override per rimuovere lo stop da qualsiasi binding"""
        kwargs.pop('stop', None)
        return super().bind(**kwargs)

def crea_llm(api_key: str, api_base: str) -> FixedAzureChatOpenAI:
    """Configurazione finale risolutiva"""
    client = CustomAzureClient(
        api_key=api_key,
        api_version="2024-08-01-preview",
        azure_endpoint=f"{api_base}/openai/deployments/gpt-4o/chat/completions",
        azure_deployment="gpt-4o"
        
    )

    return FixedAzureChatOpenAI(
        client=client,
        temperature=1,
        max_retries=3,
        streaming=False,
        model_kwargs={
            'disabled_parameters': ['stop'],
            'request_timeout': 30
        }
    )

llm = crea_llm(api_key, api_base)

# Verifica finale del client
test_client = CustomAzureClient(
    api_key=api_key,
    api_version="2024-08-01-preview",
    azure_endpoint=api_base
)

# Test diretto senza LangChain
response = test_client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "Hello"}]
)  # Funziona senza errori

c:\Users\smancone\OneDrive - BUSINESS INTEGRATION PARTNERS SPA\Desktop\progetti\2402_PRYSMIAN\.conda\Lib\site-packages\IPython\core\interactiveshell.py:3577: UserWarning: Parameters {'request_timeout'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
from typing import TypedDict, List, Optional
from langchain_community.vectorstores import Neo4jVector
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_react_agent
from neo4j.exceptions import Neo4jError
from langchain.tools import BaseTool
from langchain_core.vectorstores import VectorStoreRetriever
from pydantic import Field
from langchain_core.messages import AIMessage, HumanMessage


# Gestione dello stato dinamico
class GraphState(TypedDict):
    messages: List[dict]
    schema_data: dict
    question: str
    cypher_query: Optional[str]
    db_result: Optional[dict]

def fetch_dynamic_schema(graph: Neo4jGraph) -> dict:
    """Recupera lo schema del database Neo4j utilizzando APOC"""
    try:
        result = graph.query("""
            CALL apoc.meta.schema()
            YIELD value
            RETURN {
                nodes: [label IN keys(value) WHERE value[label].type = 'node' | {
                    label: label,
                    properties: keys(value[label].properties)
                }],
                relationships: [rel IN keys(value) WHERE value[rel].type = 'relationship' | {
                    type: rel,
                    start: value[rel].start,
                    end: value[rel].end
                }]
            } AS schema
        """)
        return result[0]["schema"] if result else {"nodes": [], "relationships": []}
    except Exception as e:
        print(f"Errore nel recupero dello schema: {str(e)}")
        return {"nodes": [], "relationships": []}

# Prompt per la generazione di Cypher
CYPHER_GENERATION_PROMPT = ChatPromptTemplate.from_messages([
    SystemMessage(content="""
    Genera query Cypher usando questo schema:
    {schema}
    
    Regole:
    1. Usa MATCH per il pattern matching
    2. Usa WITH per risultati intermedi
    3. Restituisci count() per domande numeriche
    4. Usa solo label presenti nello schema
    """),
    HumanMessage(content="{question}")
])

# Strumento per la ricerca vettoriale
class RicercaVettoriale(BaseTool):
    name: str = Field("ricerca_semantica")
    description: str = Field("Ricerca semantica per termini tecnici e concetti")
    retriever: VectorStoreRetriever = Field(exclude=True)
    
    def _run(self, query: str, **kwargs) -> list:
        return self.retriever.invoke(query, **kwargs)

# Strumento per query Cypher
class StrumentoCypher(BaseTool):
    name: str = Field("query_cypher")
    description: str = Field("Genera ed esegue query Cypher complesse sul grafo")
    graph: Neo4jGraph = Field(exclude=True)
    schema_data: dict = Field(default_factory=dict, exclude=True)
    
    def __init__(self, graph: Neo4jGraph):
        super().__init__(graph=graph)
        self.schema_data = fetch_dynamic_schema(graph)
    
    def _run(self, domanda: str) -> str:
        """Esegue la query e restituisce i risultati"""
        try:
            cypher = self._genera_cypher(domanda)
            risultato = self.graph.query(cypher)
            return str(risultato)
        except Neo4jError as e:
            return self._correggi_errore(e, cypher)
    
    """    def _genera_cypher(self, domanda: str) -> str:
        prompt_ottimizzato = CYPHER_GENERATION_PROMPT.partial(
            schema=str(self.schema_data),
            #stop_sequences=[]  # Disabilita sequenze di stop
        )
            
        catena = (
            CYPHER_GENERATION_PROMPT
            | llm#.bind(stop=None)
            | StrOutputParser()
        )
        return catena.invoke({"question": domanda})
        return catena.invoke({
            "question": domanda,
            "schema": str(self.schema_data)
        })"""
        
    def _genera_cypher(self, domanda: str) -> str:
        prompt_ottimizzato = CYPHER_GENERATION_PROMPT.partial(
            schema=str(self.schema_data)
        )
        
        catena = (
            prompt_ottimizzato
            | llm.bind(stop=None)
            | StrOutputParser()
        )
        return catena.invoke({"question": domanda})

    def _correggi_errore(self, errore: Neo4jError, cypher: str) -> str:
        correttore = (
            ChatPromptTemplate.from_template("""
            Correggi questa query Cypher:
            Errore: {errore}
            Query originale: {cypher}
            Schema: {schema}
            Query corretta:""")
            | llm
            | StrOutputParser()
        )
        cypher_corretto = correttore.invoke({
            "errore": str(errore),
            "cypher": cypher,
            "schema": str(self.schema_data)
        })
        return self.graph.query(cypher_corretto)

def format_scratchpad(steps):
    """Convert intermediate steps to proper message format"""
    messages = []
    for action, observation in steps:
        messages.append(AIMessage(content=action.log))
        messages.append(HumanMessage(content=f"Observation: {observation}"))
    return messages

# Configurazione del sistema
REACT_PROMPT = ChatPromptTemplate.from_messages([
    SystemMessage(content="""Sistema esperto in analisi di grafi Neo4j
    Strumenti disponibili: {tools}
    
    Formato risposta:
    Pensiero: Analisi della domanda
    Azione: Strumento da usare
    Input Azione: Input per lo strumento
    Osservazione: Risultato ottenuto
    ...(ripeti se necessario)
    Pensiero Finale: Risposta completa
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessage(content="{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Inizializzazione vector store
vector_store = Neo4jVector.from_existing_graph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    embedding=emb_model,
    node_label="Content",
    text_node_properties=["text"],#, "metadata"],  # Multiple properties
    embedding_node_property="embedding",
    search_type="hybrid",
    keyword_index_name="content_keyword_idx"  # Separate keyword index
)

# Strumenti disponibili
tools = [
    StrumentoCypher(graph),
    RicercaVettoriale(retriever=vector_store.as_retriever())
]


# Creazione agent
agent2 = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=REACT_PROMPT.partial(
        tools="\n".join([f"- {t.name}: {t.description}" for t in tools]),
        tool_names=", ".join([t.name for t in tools])
    )
)

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=REACT_PROMPT.partial(
        tools="\n".join([f"- {t.name}: {t.description}" for t in tools]),
        tool_names=", ".join([t.name for t in tools]),
        stop_sequences=[]  # Rimuove qualsiasi riferimento a stop
    )
)

REACT_PROMPT = ChatPromptTemplate.from_messages([
    SystemMessage(content="""Sei un esperto analista di dati strutturati in Neo4j.
    Strumenti disponibili: {tools}
    Nomi strumenti: {tool_names}"""),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessage(content="{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")  # Deve essere presente
])

from langchain_core.prompts import PromptTemplate

REACT_PROMPT = PromptTemplate.from_template(
    """Answer questions using Neo4j data. Tools available: {tools}
    
    Use format:
    Question: {input}
    Thought: {agent_scratchpad}
    Action: {tool_names}
    Action Input: {{input}}
    Observation: {{result}}
    Final Answer: {{answer}}"""
)


agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=REACT_PROMPT.partial(
        tools="\n".join([f"- {t.name}: {t.description}" for t in tools]),
        tool_names=", ".join([t.name for t in tools]),
        #stop_sequence=[]
    )
)

from langchain.agents import AgentExecutor

class FixedAgentExecutor(AgentExecutor):
    def _strip_stop_params(self, inputs):
        #inputs.pop('stop', None)
        return super()._call(inputs)
    
# Esecutore
executor = FixedAgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True
)


# Esecuzione della query
risposta = executor.invoke({
    "input": "Elenca i piatti che contengono Latte+ e le loro caratteristiche",
    "chat_history": [],
    #"agent_scratchpad": format_scratchpad(response.get("intermediate_steps", []))
})

print(risposta['output'])




> Entering new FixedAgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': "Unsupported parameter: 'stop' is not supported with this model.", 'type': 'invalid_request_error', 'param': 'stop', 'code': 'unsupported_parameter'}}

In [10]:
print(inspect.getsource(llm.client.create))


    def create(self, *args, **kwargs):
        kwargs.pop('stop', None)  # Rimuove 'stop' prima della chiamata
        return super().create(*args, **kwargs)



In [12]:
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x0000023D0897C290> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000023D08A50AD0> root_client=<openai.lib.azure.AzureOpenAI object at 0x0000023D07C10110> root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x0000023D03339850> temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********') disabled_params={'parallel_tool_calls': None} azure_endpoint='https://ai-kms-openai2.openai.azure.com/openai/deployments/o1-mini/chat/completions?api-version=2024-08-01-preview' deployment_name='o1-mini' openai_api_version='2024-08-01-preview' openai_api_type='azure'


# Test 4


In [ ]:
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_openai import ChatOpenAI

print(graph.schema)

Node properties:
Tecnica {nome: STRING, Gravità_Psionica: STRING, Livello: STRING, tipo: STRING, caratteristica: STRING, version: STRING, livello: STRING, uso: STRING, applicazione: STRING, effetto: STRING, Termocinetica_Multipla: STRING, Frugale_Energeticamente_Negativa: STRING, Quantum_Fluttuante: STRING, Flusso: STRING, Frugale: STRING, Dinamico: STRING, ElettroOsmotica: STRING, Metodo: STRING, cabina: STRING, qualita: STRING, gravita: STRING, descrizione: STRING, Descrizione: STRING, Infusione: STRING, Svantaggi: STRING, Vantaggi: STRING, Funziona: STRING, licenza: STRING, vantaggi: STRING, svantaggi: STRING, funzione: STRING, fondamentali: STRING, funziona: STRING, Tipo: STRING, Tecnologia: STRING, Spaziatura: STRING, Qualità: STRING, ingredienti: STRING, moduli_elettro_osmotici: STRING, sequenze_innesto: STRING, distribuzione_idrica: STRING, sfera_onirico_mistica: STRING, matrice_cosmica: STRING, narrazione: STRING, campi_energetici: STRING, calore_residuo: STRING, sincronizzazio

In [31]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

from langchain_core.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{name:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, return_intermediate_steps=True, allow_dangerous_requests=True, cypher_prompt=CYPHER_GENERATION_PROMPT,
)

result = chain.invoke({"query": "Quali sono i piatti della galassia che contengono Latte+"})
print(f"Intermediate steps: {result['intermediate_steps']}")
print(f"Final answer: {result['result']}")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Piatto)-[:CONTIENE]->(:Ingrediente {nome: "Latte+"})
RETURN p.nome AS nomePiatto

Full Context:
[{'nomePiatto': "Pizza Cosmica all'Essenza di Drago con Nebbia Arcobaleno e Funghi Orbitali"}]

> Finished chain.
Intermediate steps: [{'query': 'cypher\nMATCH (p:Piatto)-[:CONTIENE]->(:Ingrediente {nome: "Latte+"})\nRETURN p.nome AS nomePiatto\n'}, {'context': [{'nomePiatto': "Pizza Cosmica all'Essenza di Drago con Nebbia Arcobaleno e Funghi Orbitali"}]}]
Final answer: Non conosco i piatti che contengono Latte+.
